<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Deep Learning</center>
# Worksheet S2: Redes Neuronales Convolucionales (CNNs)

#### https://www.youtube.com/watch?v=2-Ol7ZB0MmU
#### https://www.youtube.com/watch?v=FTr3n7uBIuE
#### https://github.com/llSourcell/Convolutional_neural_network

#Lección Siraj
http://localhost:8889/notebooks/Documents/DataScience/Convolutional_neural_network-master/Convolutional_neural_network-master/convolutional_network_tutorial.ipynb

## Objetivos

El objetivo de este worksheet es comprender las características principales de las redes convolucionales así como su implementación en TensorFlow

## Introducción

Las redes neuronales convolucionales están presentes en la gran mayoria de algoritmos del estado del arte en Machine Learning hoy en día. Son una tipo de red neuronal donde las neuronas corresponden a campos receptivos, de forma muy parecida a las neuronas en la corteza visual primaria de un cerebro humano. Desde un punto de vista técnico, son una variación de las redes neuronales profundas (perceptrón multicapa), donde las neuronas se aplican a matrices bidimensionales. Por este motivo, son tremendamente efectivas en tareas de visión artificial, como la clasificación y/o segmentación de imagenes o videos.

En la actualidad, las redes neuronales convolucionales pueden trabajar con arrays de 1D (señales o secuencias), 2D (imágenes) o 3D (video). Son el estado del arte en innumerables problemas como el reconocimiento de objetos o la transcripción de escritura manuscrita.


## Partes de una Red Neuronal Convolucional

A continuación vamos a desgranar las distintas partes de una red neuronal convolucional para entender su funcionamiento.

En primer lugar, debemos saber que a lo que llamamos una capa de una CNN está formado por dos sub-capas que son la sub-capa convolucional y la sub-capa de pooling o subsampling.

### Capa convolucional

La capa convolucional puede entenderse como un extractor de características. La capa convolucional aplica un número de filtros de convolución a la 'imagen' de entrada, creando una serie de mapas de carácteristicas aplicando filtros. Una característica que es muy importante comprender es que cada mapa de características, está formado aplicando un mismo filtro compartido a distintas partes de la imagen de entrada. 

<img src="cnn_01.png" style="height: 400px">

Como podemos ver en la imagen, la sub-capa convolucional está formada por:

- Una entrada (para nuestro ejemplo será una imagen)
- Campo receptivo, es el tamaño de la entrada que se procesara para extraer una característica
- Matriz de pesos, contiene todos los filtros que se van a utilizar en esta capa para extraer características

Suponiendo que nuestra capa convolucional tiene un sólo filtro, lo que hacemos es ir recorriendo nuestra imagen de entrada poco a poco, de forma que extraeremos un punto de nuestro feature map mediante la aplicación de una función no lineal a la convolución de nuestro campo receptivo con nuestro filtro, produciendo un sólo valor en el mapa de características por cada sub-región de la imagen de entrada a la que aplicamos el filtro.

Como vemos en las imagenes siguientes, lo que vamos haciendo es símplemente mover este campo receptivo poco a poco por toda la imagen de entrada hasta que la hemos recorrido por completo, obteniendo un mapa de características.

<img src="cnn_02.png" style="height: 400px">
<img src="cnn_03.png" style="height: 400px">

En la práctica, nuestra sub-capa convolucional tendrá varios filtros, por lo que este proceso se repite para cada uno de ellos, obteniendo así varios mapas de características.

<img src="cnn_04.png" style="height: 400px">


### Capa de agrupación (pooling)

A cada convolucional le suele seguir una capa pooling, que se encarga de reducir la dimensionalidad de la imagen extraída por la capa convolucional o mapa de características. Esto se hace para reducir el tiempo de procesado necesario y para obtener cierta invariabilidad a pequeñas rotaciones o traslaciones. 

Esta capa realiza una operación recorriendo poco a poco la imagen de entrada (que es la salida de la capa convolucional). De forma similar a lo que hacía la capa anterior, irá mirando a un campo receptivo y realizando una operación sencilla, que suele ser una media de todos los valores o una selección del máximo. Una posibilidad sería un campo receptivo de 2x2 pixels y función max, con esta configuración esta capa iría procesando la imagen de entrada en regiones de 2x2 y seleccionando el máximo de cada una de estas regiones.

<img src="cnn_05.png" style="height: 400px">


La arquitectura más típica de una red convolucional consiste en una serie de módulos convolucionales que actúan como extractor de características. Cada uno de estos módulos está formado por una capa convolucional seguido de una capa pooling. A continuación, se utiliza una o varias capas perceptron, que llamaremos Dense, que realiza la clasificación final. En un problema de clasificación multiclase la última capa será de tipo Dense y tendrá tantas neuronas como clases tenga el problema, y la función de activación será de tipo softmax.

La función de activación softmax fuerza a que la suma de todas las salidas sea 1, de forma que las salidas puedan interpretarse como la probabilidad de la imagen de entrada de pertenecer a cada una de esas clases.

En la siguiente imagen podemos ver una posible estructura para un problema de reconocimiento de idioma, cuya entrada es un espectrograma y hay 8 posibles idiomas.

<img src="cnn_06.png" style="height: 400px">

*MI NOTA: en general, el número de neuronas de la capa de entrada será igual a el numero de features (dimensión que será largo x ancho)*

*MI NOTA: por ejemplo la mapa de aristas sería como una matriz de 0 y 1 donde cada pixel es parte o no de una arista (de esto va el tema de filtros creo)*

*MI NOTA: el número de filtros lo especificamos nosotros (por ensayo-error, esto es "oculto" en cierto modo)*

*MI NOTA: la última dibujada en rojo es aplicando la función softmax*

## CNNs en TensorFlow

Ahora vamos a ver cómo se definen estas capas en TensorFlow, como siempre, comenzamos importando los paquetes que vamos a necesitar.

Además, importaremos los datos que vamos a utilizar, correspondientes a MNIST, definirmeos también algunas variables de los datos de entrada e inicializaremos los placeholder que almacenaran nuestros datos.

In [1]:
from __future__ import print_function
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True) 
#one_hot es un vector de todo ceros salvo un 1; vector tamaño 10 que indica qué cifra es (en la posición correcta será 1)

# Parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 128 #cada ctos datos individuales doy un paso (al paso de learning rate)
display_step = 10 

# Network Parameters
n_input = 784   # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


A continuación vamos a ver cada una de estas capas:

### Capa de entrada

En primer lugar, necesitamos modificar el formato de entrada de los datos, ya que las capas convolucionales esperan tensores de 4 dimensiones:  #muestras x ancho x alto x #canales

En nuestro caso, vamos a realizar la redefinición sobre el propio placeholder que contiene los datos. Nuestras imágenes son de tamaño 28x28 y al ser monocromáticas el número de canales será 1:

In [2]:
x_reshaped = tf.reshape(x, shape=[-1, 28, 28, 1])  #shape --> batch, ancho, alto, canales

Como podemos ver, la primera dimensión, el número de muestras, la hemos definido con valor -1, esto indica a TensorFlow que esta dimensión debe ser calculada de forma dinámica en tiempo de ejecución, ya que aún no sabemos cuantas muestras tendremos.

### Convolucional

Utilizaremos la función conv2d(). Crea una capa convolucional para entradas de 2 dimensiones. 

Recibe como entrada:
- inputs, que es la entrada a la capa
- filters, que es el número de filtros que se va a utilizar
- kenel_size, que es el tamaño de los filtros
- strides, que es el paso que se va a utilizar entre campo receptivo y campo receptivo
- padding, que es para el relleno a la entrada, nosotros utilizaremos ¨SAME¨
- activation, que es la funcion que se va a aplicar después de la convolución


In [4]:
input_layer = tf.reshape(x, [-1, 28, 28, 1])

conv1 = tf.layers.conv2d(
    inputs=input_layer,
    filters=32, #num filtros
    kernel_size=[5,5],  #tamaño del filtro
    strides=(1, 1), #cada cuántos pixeles se mueve (creo)
    activation=tf.nn.relu,
)

# conv1 = tf.layers.conv2d(input_layer, 32, 5, activation=tf.nn.relu)

Como podemos ver, conv2d espera un tensor de 4 dimensiones: #muestras, ancho, alto, #canales

La salida será otro tensor de 4 dimensiones.

### Pooling

Utilizaremos la función max_pooling2d(). Esta función realiza la función max sobre el campo receptivo que le indiquemos, en nuestro caso utilizaremos un filtro de tamaño 2x2 con un paso entre muestreos de 2.

Recibe como entrada:
- inputs, que es la entrada a la capa
- pool_size, que es el tamaño del campo receptivo
- strides, que es el paso al hacer el barrido de la imagen de entrada


In [5]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

### Dense

Finalmente, utilizaremos una capa de tipo Dense. Para ello primero vamos a convertir nuestra imagen 2D (la salida de pool1) en un vector, para ello utilizaremos la función flatten()

A continuacion utilizamos la función tf.layers.dense(), que recibe como entrada el número de unidades y la función de activación, la salida será del tamaño del número de unidades.

In [6]:
pool1_flat = tf.contrib.layers.flatten(pool1)  #vectorizamos la entrada (vector de nx1)

dense = tf.layers.dense(inputs=pool1_flat, units=512, activation=tf.nn.relu)  #units-> num de neuronas, esto es capa oculta y pto

* mi nota: dense, feedforward, DNN, recurrente, etc... tipos de capas*

### Capa de salida

Finalmente, necesitamos una capa de salida, que será de tipo Dense y tendrá tantas neuronas como clases tiene nuestro problema.

La función de activación será de tipo softmax, de esta forma la salida pueda interpretarse como probabilidades de pertenecer a cada una de las clases.


In [7]:
output = tf.layers.dense(inputs=dense, units=n_classes, activation=tf.nn.softmax)

### Englobando la creación del modelo en una función por simplicidad:

Previamente hemos visto qué instrucciones necesitamos para crear cada una de las capas que necesitaremos en nuestro modelo. A continuación, vamos a unir todos estos comandos en una sola función para poder llamarla cuando creemos el grafo computaciónal de forma más simple y ordenada. Esto nos permite también crear distintos modelos, uno en cada función, y poder invocar uno u otro en tiempo de ejecución:

In [8]:
def conv_net(x, n_classes):

    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer with 32 filters and a kernel size of 5
    conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
    # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

    # Flatten the data to a 1-D vector for the fully connected layer
    fc1 = tf.contrib.layers.flatten(conv1)

    # Fully connected layer (in tf contrib folder for now)
    fc1 = tf.layers.dense(fc1, 1024)  #aquí por ej. en lugar de 512 hemos puesto 1024 units
        
    # Output layer, class prediction
    out = tf.layers.dense(fc1, n_classes)

    return out


### Construyendo el grafo computacional

Una vez que tenemos nuestro modelo definido en una función, podemos pasar a la construcción del grafo computacional completo, que en este caso se compone del modelo, de la función de coste a minimizar, del optimizador, y de las funciones necesarias para evaluar el modelo.

Pred contendrá nuestro modelo, que al recibir los datos como entrada proporciona la probabilidad de pertenencia a cada una de las clases.

Cost será nuestra función de coste, en este caso hemos utilizado entropía cruzada que podemos interpretarlo como una distancia entre la etiqueta y la predicción, más información en: https://en.wikipedia.org/wiki/Cross_entropy

Optimizer va a ser nuestro optimizador, en este caso vamos a utilizar Adam, se utiliza de la misma forma que el descenso por gradiente y es una versión mejorada de este.

Correct pred tomará un valor de 1 cuando la predicción sea correcta y un valor de 0 cuando sea incorrecta.

Accuracy será el porcentaje de aciertos de la red.

In [10]:
# Construct model
pred = conv_net(x, n_classes)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  #aquí Adam, pero podría ser SGD por ej

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Ejecutando el grafo computacional y evaluando el resultado

Por último, inicializaremos las variables globales, y definiremos una sesión. Dentro de esta sesión realizaremos tantas iteraciones como hayamos definido al principio de nuestro código y en cada una de estas iteraciones ejecutaremos nuestro modelo con el optimizador seleccionado.

Como podmeos ver en el código, los pasos son los mismos que seguimos con el regresor lineal, con las siguientes modificaciones:

- El número de iteraciones no es fijo, depende del tamaño del batch.
- Hemos añadido una función para imprimir por pantalla el rendimiento actual cada display_step pasos

En la parte final, ejecutamos el grafo computacional ya entrenado con los datos de test y mostramos el resultado.

In [14]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              })
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      }))

Iter 1280, Minibatch Loss= 0.620316, Training Accuracy= 0.86719
Iter 2560, Minibatch Loss= 0.279208, Training Accuracy= 0.91406
Iter 3840, Minibatch Loss= 0.218876, Training Accuracy= 0.91406
Iter 5120, Minibatch Loss= 0.289511, Training Accuracy= 0.92969
Iter 6400, Minibatch Loss= 0.182966, Training Accuracy= 0.94531
Iter 7680, Minibatch Loss= 0.205121, Training Accuracy= 0.94531
Iter 8960, Minibatch Loss= 0.162494, Training Accuracy= 0.95312
Optimization Finished!
Testing Accuracy: 0.96484375


## Resumen del código utilizado

A modo de resumen, se muestra a continuación el código que se ha utilizado, y que será el punto de partida para el challenge de esta clase:

In [19]:
from __future__ import print_function
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_iters = 10000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784   # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

def conv_net(x, n_classes):

    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer with 32 filters and a kernel size of 5
    conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
    # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

    # Flatten the data to a 1-D vector for the fully connected layer
    fc1 = tf.contrib.layers.flatten(conv1)

    # Fully connected layer (in tf contrib folder for now)
    fc1 = tf.layers.dense(fc1, 1024)
        
    # Output layer, class prediction
    out = tf.layers.dense(fc1, n_classes)
    
    return out

# Construct model
pred = conv_net(x, n_classes)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              })
            
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      }))
    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Iter 1280, Minibatch Loss= 0.634154, Training Accuracy= 0.80469
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Iter 2560, Minibatch Loss= 0.537672, Training Accuracy= 0.85938
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Iter 3840, Minibatch Loss= 0.250504, Training Accuracy= 0.91406
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0.